In [1]:
import taichi as ti
from ipycanvas import Canvas
from IPython.display import display
import ipywidgets as widgets
import time
import numpy as np
import signalflow as sf
from PIL import Image

# Initialize Taichi
ti.init(arch=ti.cpu)

n = 500
pixels = ti.Vector.field(3, dtype=float, shape=(n, n))  # RGB field for colors
bg = ti.Vector.field(3, dtype=float, shape=(n, n))  # Background image
magnifier = ti.Vector.field(3, dtype=float, shape=(n, n))  # Magnifier

# Cursor position (normalized to [0, 1]) and mouse state
cursor_pos = ti.Vector.field(2, dtype=float, shape=())
mouse_pressed = ti.field(dtype=int, shape=())  # 1 if mouse button is pressed, 0 otherwise
square_size = ti.field(dtype=float, shape=())  # Size of the square (normalized)
square_stroke = 0.006  # Stroke width for the square

# read a background image into a numpy array

img_path = "/Users/balintl/Documents/GitHub/sonification/experiments/lsm_paper/figures/cellular_seq/Timepoint_001_220518-ST_C03_s1.jpg"
img = Image.open(img_path)
img = img.resize((n, n))
img_data = np.array(img)
img_data_norm = img_data / 255.0  # Normalize to [0, 1]
bg.from_numpy(img_data_norm)
bg_np = img_data


# Initialize square size
square_size[None] = 0.1

@ti.kernel
def draw_square():
    """Render a square centered at the current cursor position."""
    center_x, center_y = cursor_pos[None]
    size = square_size[None]  # Dynamic square size from slider
    center_x = ti.max(ti.min(center_x, 1.0 - size / 2), size / 2)  # Clamp to [size/2, 1-size/2]
    center_y = ti.max(ti.min(center_y, 1.0 - size / 2), size / 2)  # Clamp to [size/2, 1-size/2]
    color = ti.Vector([1.0, 0.0, 0.0]) if mouse_pressed[None] else ti.Vector([1.0, 1.0, 1.0])  # Red or white

    for i, j in pixels:
        x = i / pixels.shape[0]  # Normalize to [0, 1]
        y = j / pixels.shape[1]  # Normalize to [0, 1]
        dist_x = abs(x - center_x)
        dist_y = abs(y - center_y)
        if dist_x < size / 2 and dist_y < size / 2 and (dist_x > size/2 - square_stroke or dist_y > size/2 - square_stroke):
                pixels[i, j] = color  # Square color
        else:
            # pixels[i, j] = ti.Vector([0.0, 0.0, 0.0])  # Background (black)
            pixels[i, j] = bg[i, j]  # Background image

@ti.kernel
def draw_magnifier():
    """Render the background under the square as the magnifier image."""
    center_x, center_y = cursor_pos[None]
    size = square_size[None]
    center_x = ti.max(ti.min(center_x, 1.0 - size / 2), size / 2)
    center_y = ti.max(ti.min(center_y, 1.0 - size / 2), size / 2)

    for i, j in magnifier:
        x = i / pixels.shape[0]  # Normalize to [0, 1]
        y = j / pixels.shape[1]  # Normalize to [0, 1]
        if x < size and y < size:
            # get bg under square
            bg_x = i + int((center_x - (size / 2)) * n)
            bg_y = j + int((center_y - (size / 2)) * n)
            magnifier[i, j] = bg[bg_x, bg_y]
        else:
            magnifier[i, j] = ti.Vector([1.0, 1.0, 1.0])




graph = sf.AudioGraph(start=False)
graph.clear()

def mix2samps(mixval, eps=1e-6):
    "Convert a mix value (used in sf.Smooth) to samples"
    return np.ceil(np.log(eps) / np.log(mixval))

def samps2mix(samps, eps=1e-6):
    "Convert samples to a mix value (used in sf.Smooth)"
    return eps ** (1 / samps)

print(samps2mix(48000))

class Theremin(sf.Patch):
    def __init__(self, frequency=440, amplitude=0.5, smooth_n_samps=24000):
        super().__init__()
        frequency = self.add_input("frequency", frequency)
        amplitude = self.add_input("amplitude", amplitude)
        freq_smooth = sf.Smooth(frequency, samps2mix(smooth_n_samps))
        amplitude_smooth = sf.Smooth(amplitude, samps2mix(smooth_n_samps))
        sine = sf.SineOscillator(freq_smooth)
        output = sf.StereoPanner(sine * amplitude_smooth, pan=0)
        self.set_output(output)

theremin = Theremin()
theremin.play()


# Create a canvas
padding = 40
canvas = Canvas(width=n*2 + padding*2, height=n + padding*2)
display(canvas)

# Create a slider widget for controlling square size
slider = widgets.FloatSlider(
    value=0.1,  # Initial value
    min=0.01,  # Minimum size
    max=0.5,   # Maximum size
    step=0.01, # Step size
    description='Square Size',
)
display(slider)

audio_graph_toggle = widgets.ToggleButton(
    value=False,
    description='Audio',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Toggle audio',
    icon="speaker"
)
display(audio_graph_toggle)

# Global state variables
is_drawing = False
latest_event_time = time.time()
refresh_interval = 1 / 60  # Limit to 30 FPS

def redraw():
    """Update the canvas with the latest Taichi-generated pixels."""
    global is_drawing

    if is_drawing:
        return  # Skip if we're already drawing

    is_drawing = True
    try:
        draw_square()  # Render the square at the current cursor position
        img_data = (pixels.to_numpy() * 255).astype(np.uint8)  # Scale to [0, 255] and convert to uint8
        img_data = np.transpose(img_data, (1, 0, 2))  # Transpose to match the canvas shape
        canvas.put_image_data(img_data, padding, padding)

        draw_magnifier()  # Render the magnifier image
        magnifier_data = (magnifier.to_numpy() * 255).astype(np.uint8)  # Scale to [0, 255] and convert to uint8
        magnifier_data = np.transpose(magnifier_data, (1, 0, 2))  # Transpose to match the canvas shape
        canvas.put_image_data(magnifier_data, n+20+padding, padding)
    finally:
        is_drawing = False  # Reset drawing state

def handle_mouse_event(x, y, pressed: int = 0):
    """Update the cursor position and mouse state, and trigger a redraw."""
    global latest_event_time

    # Normalize mouse coordinates to [0, 1]
    x_clamped = np.clip(x-padding, 0, n-1)
    y_clamped = np.clip(y-padding, 0, n-1)
    cursor_pos[None] = [x_clamped / n, y_clamped / n]

    # Update mouse state
    if pressed == 2:
        mouse_pressed[None] = 1
    elif pressed == 3:
        mouse_pressed[None] = 0

    # Update synth
    # mouse_x = cursor_pos[None][0]
    square_half_size = square_size[None] / 2
    mouse_x = np.clip(x_clamped / n, square_half_size, 1 - square_half_size)
    mouse_y = 1 - np.clip(y_clamped / n, square_half_size, 1 - square_half_size)
    theremin.set_input("frequency", 440 * 2 ** (mouse_y * 4 - 2))
    theremin.set_input("amplitude", mouse_x * mouse_pressed[None])
    
    current_time = time.time()
    if current_time - latest_event_time < refresh_interval and pressed < 2:
        return  # Skip if we are processing too quickly
    latest_event_time = current_time  # Update the last event time
    
    redraw()

# Attach event listeners
canvas.on_mouse_move(lambda x, y: handle_mouse_event(x, y, 1 if mouse_pressed[None] == 1 else 0))  # Triggered during mouse movement
canvas.on_mouse_down(lambda x, y: handle_mouse_event(x, y, pressed=2))  # Mouse button pressed
canvas.on_mouse_up(lambda x, y: handle_mouse_event(x, y, pressed=3))  # Mouse button released

# Update square size from slider
def update_square_size(change):
    square_size[None] = change['new']
    redraw()

slider.observe(update_square_size, names='value')  # Observe slider changes


def toggle_audio(change):
    if change['new']:
        graph.start()
    else:
        graph.stop()

audio_graph_toggle.observe(toggle_audio, names='value')



# Initial draw
pixels.fill(0)  # Clear the field
cursor_pos[None] = [0.5, 0.5]  # Start with the square in the center
mouse_pressed[None] = 0  # Start with mouse unpressed
redraw()


[Taichi] version 1.7.3, llvm 15.0.7, commit 5ec301be, osx, python 3.10.16


[I 01/27/25 12:03:10.790 26626655] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=arm64
0.9997122182804811


[W 01/27/25 12:03:11.178 26626655] TaichiWarning
While compiling `ext_arr_to_matrix_c30_0`, File "/Users/balintl/miniconda3/envs/pixasonics/lib/python3.10/site-packages/taichi/_kernels.py", line 213, in ext_arr_to_matrix:
                        mat[I][p] = arr[I - offset, p]
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Assign may lose precision: f32 <- f64
[W 01/27/25 12:03:11.179 26626655] TaichiWarning
While compiling `ext_arr_to_matrix_c30_0`, File "/Users/balintl/miniconda3/envs/pixasonics/lib/python3.10/site-packages/taichi/_kernels.py", line 213, in ext_arr_to_matrix:
                        mat[I][p] = arr[I - offset, p]
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Assign may lose precision: f32 <- f64
[W 01/27/25 12:03:11.179 26626655] TaichiWarning
While compiling `ext_arr_to_matrix_c30_0`, File "/Users/balintl/miniconda3/envs/pixasonics/lib/python3.10/site-packages/taichi/_kernels.py", line 213, in ext_arr_to_matrix:
                        mat[I][p] = ar

Canvas(height=580, width=1080)

FloatSlider(value=0.1, description='Square Size', max=0.5, min=0.01, step=0.01)

ToggleButton(value=False, description='Audio', icon='speaker', tooltip='Toggle audio')